<a href="https://colab.research.google.com/github/jjbmsda/Speaker_Recognition/blob/main/Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Paper/FINAL/ 

/content/drive/MyDrive/Paper/FINAL


In [ ]:
!pip install matplotlib==3.3.0

     |████████████████████████████████| 11.5 MB 4.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import sklearn
import random as rn
from keras.layers import Dense
from keras import Input
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D

In [ ]:
DATA_DIR_TRAIN = 'vox1/wav/'
DATA_DIR_TEST = 'vox1_test/'

In [ ]:
Train_list = os.listdir(DATA_DIR_TRAIN)
print("Train_list = {}".format(Train_list))

Test_list = os.listdir(DATA_DIR_TEST)
print("Test_list = {}".format(Test_list))

Train_list = ['id10767', 'id10704', 'id10635', 'id10542', 'id10472', 'id10732', 'id10471', 'id10326', 'id10818', 'id10560', 'id10790', 'id10432', 'id10655', 'id10805', 'id10416', 'id10387', 'id10703', 'id10540', 'id10731', 'id10469', 'id10123', 'id10766', 'id10755', 'id10258', 'id10040', 'id10015', 'id10588', 'id10046', 'id10789', 'id10842', 'id10829', 'id10233', 'id10342', 'id10257', 'id10172', 'id10415', 'id10431', 'id10538', 'id10765', 'id10556', 'id10536', 'id10828', 'id10014', 'id10430', 'id10147', 'id10587', 'id10232', 'id10465', 'id10464', 'id10064', 'id10170', 'id10012', 'id10120', 'id10777', 'id10827', 'id10754', 'id10331', 'id10764', 'id10816', 'id10802', 'id10169', 'id10042', 'id10586', 'id10146', 'id10462', 'id10230', 'id10329', 'id10038', 'id10037', 'id10255', 'id10853', 'id10229', 'id10327', 'id10523', 'id10552', 'id10763', 'id10532', 'id10213', 'id10117', 'id10010', 'id10096', 'id10584', 'id10522', 'id10867', 'id10325', 'id10380', 'id10461', 'id10897', 'id10428', 'id1078

In [ ]:
train_X = []
train_spectrograms = []
train_mel_spectograms = []
train_mfccs = []
train_y = []

test_X = []
test_spectrograms = []
test_mel_spectograms = []
test_mfccs = []
test_y = []

pad1d = lambda a, i: a[0: i] if a.shape[0] > i else np.hstack((a, np.zeros(i-a.shape[0])))
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [ ]:
train_dataset = []

for folder in Train_list:
  fpath = librosa.util.find_files(DATA_DIR_TRAIN+'/'+folder)
  train_wav, sr = librosa.load(fpath[0])
  mfcc_train = librosa.feature.mfcc(train_wav, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
  padded_mfcc = pad2d(mfcc_train, 40)
  train_dataset.append(padded_mfcc)

print(train_dataset)  
  

[array([[-5.65143494e+02, -5.78147278e+02, -5.74466736e+02, ...,
        -5.34332642e+02, -5.46935364e+02, -5.56242188e+02],
       [ 1.41493332e+02,  1.85248215e+02,  1.84718750e+02, ...,
         2.21304306e+02,  2.16395966e+02,  2.09006470e+02],
       [ 7.69167328e+01,  4.03381271e+01,  4.94657021e+01, ...,
         3.82482986e+01,  4.62129974e+01,  4.93271828e+01],
       ...,
       [ 3.43078375e+00, -1.43539476e+00, -1.92208767e-01, ...,
        -7.26628304e-03, -1.44221282e+00,  3.06195545e+00],
       [ 5.64995956e+00,  1.04089975e-01, -6.88364506e-01, ...,
        -1.30519927e+00,  1.40088391e+00, -5.79489946e-01],
       [ 4.93609095e+00, -2.24492502e+00, -2.41832590e+00, ...,
        -1.64763689e+00, -1.69948578e+00, -4.51530647e+00]], dtype=float32), array([[-4.6965305e+02, -4.2107776e+02, -4.1344504e+02, ...,
        -2.8587750e+02, -2.8407950e+02, -2.7693634e+02],
       [ 9.8796570e+01,  1.4965781e+02,  1.3657022e+02, ...,
         2.0451331e+02,  1.9275946e+02,  2.0756

In [ ]:
test_dataset = []

for folder in Test_list:
  fpath = librosa.util.find_files(DATA_DIR_TEST+'/'+folder)
  test_wav, sr = librosa.load(fpath[0])
  mfcc_test = librosa.feature.mfcc(test_wav, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
  padded_mfcc = pad2d(mfcc_test, 40)
  test_dataset.append(padded_mfcc)

print(test_dataset) 

[array([[-5.16838989e+02, -5.44246216e+02, -5.23716248e+02, ...,
        -5.59276672e+02, -5.31817261e+02, -5.29882812e+02],
       [ 1.02180756e+02,  9.04576569e+01,  1.05638199e+02, ...,
         9.68969574e+01,  1.33115875e+02,  1.24818146e+02],
       [-7.99499130e+00, -1.91036949e+01, -2.21509438e+01, ...,
        -3.27999878e+01, -2.46455956e+01, -2.65249176e+01],
       ...,
       [ 1.02182257e+00,  8.30717087e-01, -7.47763443e+00, ...,
        -2.84054804e+00, -9.20109272e-01, -3.56342173e+00],
       [-1.17351913e+00, -5.91740513e+00, -6.50886345e+00, ...,
         5.49130440e-02, -7.04697609e-01, -1.54941845e+00],
       [-7.11235905e+00, -4.47955418e+00, -4.92078781e+00, ...,
         2.07699299e+00, -1.39438844e+00, -3.76579165e-01]], dtype=float32), array([[-5.38842224e+02, -5.45589905e+02, -5.66428528e+02, ...,
        -6.79398560e+02, -6.03987488e+02, -5.33050781e+02],
       [ 1.22074760e+02,  1.32848282e+02,  1.20984322e+02, ...,
         7.69667969e+01,  3.43303146e+

In [ ]:
import random

random.shuffle(test_dataset)
random.shuffle(train_dataset)

train_mfccs = np.array(train_dataset)
train_y = to_categorical(np.array(train_y))

test_mfccs = np.array(test_dataset)
test_y = to_categorical(np.array(test_y))

print('train_mfccs:', train_mfccs.shape)
print('train_y:', train_y.shape)

print('test_mfccs:', test_mfccs.shape)
print('test_y:', test_y.shape)

ValueError: ignored